In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Load individual datasets

In [2]:
unemp = pd.read_csv('../20_intermediate_files/unemployment_annualized_table.csv')
unemp = unemp.drop('Unnamed: 0', axis=1)
gdp = pd.read_csv('../20_intermediate_files/gdp_table.csv')
gdp = gdp.drop('Unnamed: 0', axis=1)
pop = pd.read_csv('../20_intermediate_files/population_table.csv')
pop = pop.drop('Unnamed: 0', axis=1)

### Merge datasets

In [3]:
pop_gdp = pd.merge(pop, gdp, how='inner', left_on = ['State', 'Year'], right_on = ['GeoName', 'Year'], validate='1:1', indicator=True)
pop_gdp = pop_gdp[['State', 'Year', 'Population', 'gdp', '_merge']]
pop_gdp = pop_gdp.rename(columns={"_merge": "pop_gdp_merge"})

pop_gdp['Year'] = pop_gdp['Year'].astype('str')
unemp['Year'] = unemp['Year'].astype('str')

In [4]:
df = pd.merge(unemp, pop_gdp, left_on = ['State', 'Year'], right_on = ['State', 'Year'], how='inner', validate='m:1', indicator=True)
df = df.drop(['pop_gdp_merge', '_merge'], axis=1)

### Create treatment and control states

In [5]:
# Treatment = South Dakota, West Virginia, Nebraska
# Control = Iowa, Idaho, Kansas
condition = (df['State'] == 'South Dakota') | (df['State'] == 'West Virginia') | (df['State'] == 'Nebraska')
df['treatment'] = 0
df.loc[condition, 'treatment'] = 1
df = df.reset_index(drop=True)
df['treatment'] = pd.Categorical(df['treatment']) 

### Create full dataset csv

In [6]:
df.head()

,State,Year,FIPS_Code,Civilian_Pop,Civilian_Labor_Force,Labor_Force_Pct,Employed_Total,Employed_Pct,Unemployed_Total,Unemployed_Rate,Population,gdp,treatment
0,Alabama,2010,1.0,3.690247e+06,2.196694e+06,59.508333,1.968757e+06,53.341667,227937.666667,10.375000,4785437.0,183014.5,0
1,Alabama,2011,1.0,3.725085e+06,2.201954e+06,59.116667,1.991027e+06,53.458333,210927.916667,9.583333,4799069.0,185666.9,0
2,Alabama,2012,1.0,3.745693e+06,2.179075e+06,58.166667,2.001118e+06,53.425000,177957.333333,8.158333,4815588.0,186553.9,0
3,Alabama,2013,1.0,3.765926e+06,2.172942e+06,57.725000,2.013764e+06,53.466667,159177.916667,7.325000,4830081.0,188814.2,0
4,Alabama,2014,1.0,3.783191e+06,2.164596e+06,57.225000,2.018737e+06,53.366667,145858.166667,6.733333,4841799.0,187568.0,0


In [7]:
df.to_csv('../20_intermediate_files/full_dataset.csv')

### Subset to 6 target states

In [8]:
states = ['South Dakota', 'West Virginia', 'Nebraska', 'Iowa', 'Idaho', 'Kansas']
df = df[df['State'].isin(states)]

In [9]:
df.shape

(60, 13)

### Create final dataset csv (with 6 target states)

In [10]:
df.to_csv('../20_intermediate_files/final_dataset.csv')